In [130]:
import pandas as pd
import os
from sklearn import linear_model
from sklearn.metrics import accuracy_score

current = os.getcwd() 
file1 = pd.read_csv(current + r"\..\rawdata\adult.data", header=None)
test_file = pd.read_csv(current  + r"\..\rawdata\adult.test", header=None, skiprows=1)

file = pd.concat([file1, test_file])
file.columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'salary']
file.iloc[27]



age                                54
workclass                           ?
fnlwgt                         180211
education                Some-college
education-num                      10
marital-status     Married-civ-spouse
occupation                          ?
relationship                  Husband
race               Asian-Pac-Islander
sex                              Male
capital-gain                        0
capital-loss                        0
hours-per-week                     60
native-country                  South
salary                           >50K
Name: 27, dtype: object

In [131]:
file_obj = file.select_dtypes(['object'])
file[file_obj.columns] = file_obj.apply(lambda x: x.str.strip())

In [132]:
age_contain_quest = file.loc[file['native-country'] == r'?']
age_contain_quest


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
14,40,Private,121772,Assoc-voc,11,Married-civ-spouse,Craft-repair,Husband,Asian-Pac-Islander,Male,0,0,40,?,>50K
38,31,Private,84154,Some-college,10,Married-civ-spouse,Sales,Husband,White,Male,0,0,38,?,>50K
51,18,Private,226956,HS-grad,9,Never-married,Other-service,Own-child,White,Female,0,0,30,?,<=50K
61,32,?,293936,7th-8th,4,Married-spouse-absent,?,Not-in-family,White,Male,0,0,40,?,<=50K
93,30,Private,117747,HS-grad,9,Married-civ-spouse,Sales,Wife,Asian-Pac-Islander,Female,0,1573,35,?,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15684,46,Private,31411,11th,7,Married-civ-spouse,Other-service,Wife,White,Female,0,0,40,?,<=50K.
15711,24,Private,127159,Some-college,10,Never-married,Other-service,Other-relative,White,Female,0,0,24,?,<=50K.
16043,24,State-gov,161783,Bachelors,13,Never-married,Transport-moving,Not-in-family,Black,Male,0,0,40,?,<=50K.
16090,34,Private,143776,Masters,14,Never-married,Prof-specialty,Not-in-family,Black,Male,0,0,45,?,>50K.


In [133]:
contain_question_columns = []
for column_name in file.columns:
    l = file.loc[file[column_name] == r'?'].shape[0]
    if l > 0:
        contain_question_columns.append(column_name)
        
contain_question_columns

['workclass', 'occupation', 'native-country']

In [134]:
train_set = file.loc[:, ~file.columns.isin(contain_question_columns)]

In [135]:
file.columns[file.stack().str.contains('?', regex=False).any(level=1)]

C:\Users\Acer\AppData\Local\Temp\ipykernel_17124\3724906510.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.any(level=1) should use df.groupby(level=1).any()
  file.columns[file.stack().str.contains('?', regex=False).any(level=1)]


Index(['workclass', 'occupation', 'native-country'], dtype='object')

In [136]:
import numpy as np
file.replace([r'?'], np.NaN, inplace=True)
file.replace(['>50K', '>50K.'], 0, inplace=True)
file.replace(['<=50K', '<=50K.'], 1, inplace=True)

file.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48842 entries, 0 to 16280
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             48842 non-null  int64 
 1   workclass       46043 non-null  object
 2   fnlwgt          48842 non-null  int64 
 3   education       48842 non-null  object
 4   education-num   48842 non-null  int64 
 5   marital-status  48842 non-null  object
 6   occupation      46033 non-null  object
 7   relationship    48842 non-null  object
 8   race            48842 non-null  object
 9   sex             48842 non-null  object
 10  capital-gain    48842 non-null  int64 
 11  capital-loss    48842 non-null  int64 
 12  hours-per-week  48842 non-null  int64 
 13  native-country  47985 non-null  object
 14  salary          48842 non-null  int64 
dtypes: int64(7), object(8)
memory usage: 7.0+ MB


In [137]:
file.workclass.isna()

0        False
1        False
2        False
3        False
4        False
         ...  
16276    False
16277     True
16278    False
16279    False
16280    False
Name: workclass, Length: 48842, dtype: bool

In [138]:
# m = file.workclass.mode()

# file.workclass = file.workclass.fillna(file.workclass.mode()[0])

# file.workclass
for col in contain_question_columns:
    file[col] = file[col].fillna(file[col].mode()[0])

file = pd.get_dummies(file)


In [139]:
file

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,salary,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,39,77516,13,2174,0,40,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,50,83311,13,0,0,13,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,38,215646,9,0,0,40,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,53,234721,7,0,0,40,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,28,338409,13,0,0,40,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16276,39,215419,13,0,0,36,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
16277,64,321403,9,0,0,40,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
16278,38,374983,13,0,0,50,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
16279,44,83891,13,5455,0,40,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [140]:
checkpoint = file1.shape[0]

train_data = file[:checkpoint]
test_data = file[checkpoint:]

In [141]:
train_data, test_data

(       age  fnlwgt  education-num  capital-gain  capital-loss  hours-per-week  \
 0       39   77516             13          2174             0              40   
 1       50   83311             13             0             0              13   
 2       38  215646              9             0             0              40   
 3       53  234721              7             0             0              40   
 4       28  338409             13             0             0              40   
 ...    ...     ...            ...           ...           ...             ...   
 32556   27  257302             12             0             0              38   
 32557   40  154374              9             0             0              40   
 32558   58  151910              9             0             0              40   
 32559   22  201490              9             0             0              20   
 32560   52  287927              9         15024             0              40   
 
        salary

In [142]:
train_X = train_data.loc[:, ~train_data.columns.isin(['salary'])].values
train_Y = train_data['salary'].values

test_X = test_data.loc[:, ~train_data.columns.isin(['salary'])].values
test_Y = test_data['salary'].values


model = linear_model.LogisticRegression()

model.fit(train_X, train_Y)



LogisticRegression()

In [144]:
y_pred = model.predict(test_X)

accuracy_score(test_Y, y_pred)

0.7978011178674529

In [145]:
from sklearn import tree

model = tree.DecisionTreeClassifier()

model.fit(train_X, train_Y)



DecisionTreeClassifier()

In [146]:
y_pred_tree = model.predict(test_X)

accuracy_score(test_Y, y_pred_tree)

0.8119894355383576